## INT2 Project
baseline from:
https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

In [31]:
#!unzip -q /content/gdrive/MyDrive/cifar10.zip -d /content/gdrive/MyDrive/

In [32]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
import time

In [33]:
#from torchvision.datasets.cifar import CIFAR10
TRAIN_DATA_PATH = "/content/gdrive/MyDrive/cifar10/train"
TEST_DATA_PATH = "/content/gdrive/MyDrive/cifar10/test"

transform = transforms.Compose(
    [transforms.AutoAugment(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#transform = transforms.Compose(
#    [transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


deer  bird  cat   truck


In [34]:

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imgGrid = torchvision.utils.make_grid(images)
imshow(imgGrid)
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [35]:
writer = SummaryWriter("runs/new-VGG16") #good one?
#tensorboard --logdir runs/testValidationError
writer.add_image("four images", imgGrid)
writer.flush()

In [36]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1_1 = nn.Conv2d(3, 32, 3, padding="same")
        self.conv1_2 = nn.Conv2d(32, 32, 3, padding="same")
        self.drop1 = nn.Dropout(0.2)
        self.conv2_1 = nn.Conv2d(32, 64, 3, padding="same")
        self.conv2_2 = nn.Conv2d(64, 64, 3, padding="same")
        self.drop2 = nn.Dropout(0.3)
        self.conv3_1 = nn.Conv2d(64, 128, 3, padding="same")
        self.conv3_2 = nn.Conv2d(128, 128, 3, padding="same")
        self.drop3 = nn.Dropout(0.4)
        self.conv4_1 = nn.Conv2d(128, 256, 3, padding="same")
        self.conv4_2 = nn.Conv2d(256, 256, 3, padding="same")
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(2048, 10)
        #self.fc2 = nn.Linear(1024, 512)
        #self.fc3 = nn.Linear(512, 10)

        self.sm1 = nn.Softmax()

    def forward(self, x):
        x = F.relu(self.conv1_1(x))
        x = self.pool(F.relu(self.conv1_2(x)))
        x = self.drop1(x)
        x = F.relu(self.conv2_1(x))
        x = self.pool(F.relu(self.conv2_2(x)))
        x = self.drop2(x)
        x = F.relu(self.conv3_1(x))
        #x = F.relu(self.conv3_2(x))
        x = self.pool(F.relu(self.conv3_2(x)))
        x = self.drop3(x)
        #x = F.relu(self.conv4_1(x))
        #x = F.relu(self.conv4_2(x))
        #x = F.relu(self.conv4_3(x))
        #x = self.pool(F.relu(self.conv4_2(x)))

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = self.sm1(self.fc1(x))
        #x = self.fc3(x)
        return x


net = Net()

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
print(len(testloader))
print(len(trainloader))
start_time = time.time()
for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 1000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.3f} time: {time.time() - start_time}')
            # Check against the validation set
            running_vloss = 0.0

            net.train(False) # Don't need to track gradents for validation
            for j, vdata in enumerate(testloader, 0):
                vinputs, vlabels = vdata
                voutputs = net(vinputs)
                vloss = criterion(voutputs, vlabels)
                running_vloss += vloss.item()
            net.train(True) # Turn gradients back on for training

            avg_loss = running_loss / 1000
            avg_vloss = running_vloss / len(testloader)

            # Log the running loss averaged per batch
            writer.add_scalars('Training vs. Validation Loss',
                            { 'Training' : avg_loss, 'Validation' : avg_vloss },
                            epoch * len(trainloader) + i)
            running_loss = 0.0
    print(f'epoch time: {time.time() - start_time}')

print('Finished Training')

2500
12500
[1,  1000] loss: 2.304 time: 17.755096197128296
[1,  2000] loss: 2.303 time: 50.76529598236084
[1,  3000] loss: 2.302 time: 80.29887795448303
[1,  4000] loss: 2.294 time: 107.12784242630005
[1,  5000] loss: 2.259 time: 133.03243780136108
[1,  6000] loss: 2.209 time: 158.9088683128357
[1,  7000] loss: 2.166 time: 185.86078882217407
[1,  8000] loss: 2.127 time: 213.80806303024292
[1,  9000] loss: 2.056 time: 239.8982424736023
[1, 10000] loss: 1.998 time: 265.11901569366455
[1, 11000] loss: 1.955 time: 290.8578910827637
[1, 12000] loss: 1.907 time: 317.51991081237793
epoch time: 337.2973334789276
[2,  1000] loss: 1.857 time: 354.9398195743561
[2,  2000] loss: 1.823 time: 382.5639822483063
[2,  3000] loss: 1.768 time: 407.9395627975464
[2,  4000] loss: 1.721 time: 434.0355477333069
[2,  5000] loss: 1.699 time: 459.5828278064728
[2,  6000] loss: 1.684 time: 485.90167474746704
[2,  7000] loss: 1.635 time: 512.9189102649689
[2,  8000] loss: 1.618 time: 539.5335421562195
[2,  9000] 

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')